In [8]:
import math
import xfacpy as xfac


def price_trinomial_call_ttcross(
    N,
    S0,
    K,
    r,
    T,
    u, m, d,
    pu, pm, pd,
    bond_dim=50,
    reltol=1e-8,
    max_sweeps=20,
    verbose=False,
):
    """
    Pricing di una call europea via trinomial tree + TT-cross (xfac).

    f(path) = E[discounted payoff] discreto, cioè
    f(indices) = e^{-rT} * Prob(path) * (S_T(path) - K)^+
    e poi facciamo direttamente sum1() sul TT.
    """

    if abs((pu + pm + pd) - 1.0) > 1e-10:
        raise ValueError("Le probabilità pu, pm, pd devono sommare a 1.")

    # Controllo minimo: il path tutto-up deve essere in-the-money,
    # altrimenti pivot1 non è un pivot "buono"
    ST_all_up = S0 * (u ** N)
    if ST_all_up <= K:
        raise ValueError(
            f"Il path tutto-up è ancora out-of-the-money (S_T={ST_all_up} <= K={K}). "
            "Scegli parametri diversi per u/S0/K oppure un pivot diverso."
        )

    disc = math.exp(-r * T)
    localDim = [3] * N

    def f(indices):
        S = S0
        prob = 1.0
        for move in indices:
            if move == 0:
                S *= d
                prob *= pd
            elif move == 1:
                S *= m
                prob *= pm
            else:  # move == 2
                S *= u
                prob *= pu
        payoff = max(S - K, 0.0)
        return disc * prob * payoff

    # Parametri TT-cross
    param = xfac.TensorCI2Param()
    param.bondDim = bond_dim
    param.reltol  = reltol

    # Scegliamo pivot1 = path tutto-up: [2,2,...,2]
    pivot = [2] * N
    p1 = param.pivot1
    # Cerco di capire se è una lista di interi o lista di liste
    if isinstance(p1, list) and len(p1) > 0 and isinstance(p1[0], list):
        # caso MultiIndex-like (lista di multiindici)
        param.pivot1 = [pivot]
    else:
        # caso semplice: un solo multiindice
        param.pivot1 = pivot

    # Costruzione CI2
    ci = xfac.TensorCI2(f, localDim, param)

    if verbose:
        print("Errore iniziale (trueError):", ci.trueError())

    for k in range(max_sweeps):
        ci.iterate(1)
        if verbose:
            print(f"Iter {k+1}, trueError =", ci.trueError())
        if ci.isDone():
            if verbose:
                print("Convergenza raggiunta (isDone=True).")
            break

    tt = ci.tt

    # Ora la somma di f su tutti i path è semplicemente:
    price = tt.sum1()
    return price



In [9]:
import math
import itertools

def price_trinomial_call_bruteforce(
    N,
    S0,
    K,
    r,
    T,
    u, m, d,
    pu, pm, pd,
):
    disc = math.exp(-r * T)
    total = 0.0
    for path in itertools.product([0, 1, 2], repeat=N):
        S = S0
        prob = 1.0
        for move in path:
            if move == 0:
                S *= d
                prob *= pd
            elif move == 1:
                S *= m
                prob *= pm
            else:
                S *= u
                prob *= pu
        payoff = max(S - K, 0.0)
        total += disc * prob * payoff
    return total


In [10]:
N = 5
S0 = 100.0
K  = 100.0
r  = 0.05
T  = 1.0

u  = 1.1
m  = 1.0
d  = 0.9
pu = 0.3
pm = 0.4
pd = 0.3

exact = price_trinomial_call_bruteforce(N, S0, K, r, T, u, m, d, pu, pm, pd)
tt_price = price_trinomial_call_ttcross(
    N, S0, K, r, T, u, m, d, pu, pm, pd,
    bond_dim=50, reltol=1e-10, max_sweeps=30,
    verbose=True,
)

print("Prezzo brute force =", exact)
print("Prezzo TT-cross    =", tt_price)


Errore iniziale (trueError): 0.15096349673614365
Iter 1, trueError = 0.04137070789262364
Iter 2, trueError = 0.03657286891320353
Convergenza raggiunta (isDone=True).
Prezzo brute force = 6.597107638465255
Prezzo TT-cross    = 6.6654768280158025
